In [1]:
from pytao import Tao

In [2]:
tao = Tao(f'-init /global/homes/w/wlou1991/cfs/GitHub/lcls-lattice/bmad/models/cu_spec/tao.init -noplot')

In [6]:
%%tao
sho lat * -at alias

-------------------------
Tao> sho lat * -at alias
# Values shown are for the Exit End of each Element:
# Index  name      key                      s       l  alias
      0  BEGINNING Beginning_Ele        0.000     ---
      1  DL00      Drift               -0.869  -0.869
      2  LOADLOCK  Drift                0.000   0.869
      3  BEGGUN    Marker               0.000   0.000
      4  SOL1BK    Solenoid             0.000   0.000  SOLN:IN20:111
      5  DBMARK80  Marker               0.000   0.000
      6  CATHODE   Instrument           0.000   0.000  CATH:IN20:111
      7  DL01A     Drift                0.096   0.096
      8  SOL1#1    Solenoid             0.196   0.100
      9  YC00      Vkicker              0.196   0.000  YCOR:IN20:122
     10  XC00      Hkicker              0.196   0.000  XCOR:IN20:121
     11  SQ01      Multipole            0.196   0.000  QUAD:IN20:122
     12  CQ01      Multipole            0.196   0.000  QUAD:IN20:121
     13  SOL1#2    Solenoid             0.2

In [5]:
%%tao
sho ele QE04

-------------------------
Tao> sho ele QE04
Element # 219
Element Name: QE04
Element Type:  "ETA"
Element Alias:  "QUAD:IN20:525"
Key: Quadrupole
S_start, S:     11.923644,     12.031644
Ref_time:  4.015484E-08

Attribute values [Only non-zero values shown]:
    1  L                           =  1.0800000E-01 m
    4  K1                          = -6.7899098E+00 1/m^2    45  B1_GRADIENT                 =  3.0575527E+00 T/m
   10  FRINGE_TYPE                 =  None (1)               11  FRINGE_AT                   =  Both_Ends (3)
   13  SPIN_FRINGE_ON              =  T (1)
   47  PTC_CANONICAL_COORDS        =  T (1)
   50  DELTA_REF_TIME              =  3.6025180E-10 sec
   53  P0C                         =  1.3499903E+08 eV           BETA                        =  9.9999284E-01
   54  E_TOT                       =  1.3500000E+08 eV           GAMMA                       =  2.6418841E+02
   65  INTEGRATOR_ORDER            = 2
   67  DS_STEP                     =  3.0000000E-03 m       

In [12]:
from pmd_beamphysics import ParticleGroup, particle_paths
from pmd_beamphysics.interfaces.elegant import elegant_to_data
from h5py import File

import os
import numpy as np

from pytao import Tao

root_dir = os.getcwd()

# Tao

In [5]:
# Make a bmad beam via ParticleGroup
# Note that these particles aim to follow the Bmad coordinate ( not at rest )

Q = 13E-12     # Total charge
Np = 13     # Number of macro particles
data = {'x':np.array([0,-1E-3,1E-3, 0,0,0,0,0,0,0,0,0,0]),
        'px':np.array([0,0,0,-1E-3,1E-3, 0,0,0,0,0,0,0,0]),
        'y':np.array([0,0,0,0,0,-1E-3,1E-3, 0,0,0,0,0,0]),
        'py':np.array([0,0,0,0,0,0,0,-1E-3,1E-3, 0,0,0,0]),
        'z':np.zeros(Np),
        'pz':np.array([0,0,0,0,0,0,0,0,0,0,0,-1E-3,1E-3]),
        't': np.array([0,0,0,0,0,0,0,0,0,-1E-3,1E-3, 0,0]),
        'status':np.ones(Np), 
        'weight':np.ones(Np)*Q/Np, 
        'species':'electron'}
P = ParticleGroup(data=data)

P.write_bmad('bmad/beam_13p_bmad.txt')
#P.write_elegant('elegant/beam_13p_elegant.sdds')

In [6]:
# Run Bmad (loading in the beam without tracking) to produce the corresponding beam file in hdf5 (with correct coordinate )
# Load the hdf5 file into ParticleGroup again to produce the corresponding beam file for Elegant

tao = Tao(f'-init ./bmad/tao.init -noplot')

tao.cmd("beamon") # No tracking is required here...

[]

In [17]:
%%tao
write beam beam0_13p.hdf5 -at BEGINNING   ! For elegant use 

write beam ./bmad/beam_end_tao.hdf5 -at END   ! For analysis

-------------------------
Tao> write beam beam0_13p.hdf5 -at BEGINNING   ! For elegant use 
[INFO] tao_write_cmd:
    Written: beam0_13p.hdf5
-------------------------
Tao> 
-------------------------
Tao> write beam ./bmad/beam_end_tao.hdf5 -at END   ! For analysis
[INFO] tao_write_cmd:
    Written: ./bmad/beam_end_tao.hdf5
-------------------------
Tao> 


In [15]:
H5FILE = root_dir+'/beam0_13p.hdf5'
P = ParticleGroup(H5FILE)
P.write_elegant(root_dir + '/elegant/beam0_13p.sdds')

# Tao

In [34]:
tao.lat_list('END', 'ele.a.beta')

array([9.92226266])

In [35]:
tao.lat_list('END', 'ele.a.eta')

array([0.])

In [36]:
def get_twiss(tao, ele):
    t1 = tao.lat_list(ele, 'beta_a')[0]
    t2 = tao.lat_list(ele, 'alpha_a')[0]
    t3 = tao.lat_list(ele, 'beta_b')[0]
    t4 = tao.lat_list(ele, 'alpha_b')[0]
    t5 = tao.lat_list(ele, 'eta_x')[0]
    t6 = tao.lat_list(ele, 'etap_x')[0]
    t7 = tao.lat_list(ele, 'eta_y')[0]
    t8 = tao.lat_list(ele, 'etap_y')[0]
    return t1, t2, t3, t4, t5, t6, t7, t8

In [37]:
get_twiss(tao, 'END')

(9.922262663518694,
 1.589689616045681,
 9.958118884693812,
 0.39687469868158354,
 0.0,
 0.0,
 0.0,
 0.0)

In [19]:
H5FILE = root_dir+'/bmad/beam_end_tao.hdf5'
P = ParticleGroup(H5FILE)

In [20]:
P.x

array([ 0.00000000e+00, -1.00090014e-03,  1.00090014e-03, -3.00090008e-05,
        3.00090008e-05,  0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
        0.00000000e+00,  0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
        0.00000000e+00])

In [13]:
%%tao
write beam beam0_13p.hdf5 -at BEGINNING

-------------------------
Tao> write beam beam0_13p.hdf5 -at BEGINNING
[INFO] tao_write_cmd:
    Written: beam0_13p.hdf5
-------------------------
Tao> 


In [14]:
!pwd

cmake(8):ERROR:105: Unable to locate a modulefile for 'cmake/3.18.2'
/global/u2/w/wlou1991/bmad_vs_elegant_simple


In [15]:
# Load
#h5 = File(H5FILE, 'r')
#ppaths = particle_paths(h5)
#ph5 = h5[ppaths[0]]

P = ParticleGroup(H5FILE)

#P.write_elegant('beam_BEAM0_1001_nopz.sdds')
#P.write_elegant(str(locname) + '.sdds')
#P.write_elegant('beam_BEAM0_Np202_ZPZcross_XPX.sdds')
#P.write_elegant('beam_BEAM0_Np202_XPX_100MeV.sdds')

#.write_elegant('Glen_beam_Np1M.sdds')
P.write_elegant('beam0_13p.sdds')

In [21]:
%%tao
sho ele 1

-------------------------
Tao> sho ele 1
Element # 1
Element Name: Q1
Key: Quadrupole
S_start, S:      0.000000,      0.030000
Ref_time:  1.002001E-10

Attribute values [Only non-zero values shown]:
    1  L                           =  3.0000000E-02 m
    4  K1                          =  2.0000000E+00 1/m^2    45  B1_GRADIENT                 =  6.6625662E-02 T/m
   10  FRINGE_TYPE                 =  None (1)               11  FRINGE_AT                   =  Both_Ends (3)
   13  SPIN_FRINGE_ON              =  T (1)
   47  PTC_CANONICAL_COORDS        =  T (1)
   50  DELTA_REF_TIME              =  1.0020014E-10 sec
   53  P0C                         =  9.9869355E+06 eV           BETA                        =  9.9869355E-01
   54  E_TOT                       =  1.0000000E+07 eV           GAMMA                       =  1.9569512E+01
   65  INTEGRATOR_ORDER            = 2
   67  DS_STEP                     =  1.0000000E-02 m        66  NUM_STEPS                   = 3
   68  CSR_DS_STEP     

In [20]:
1.954394518728e+01 * 0.511e6

9986955.99070008

In [ ]:
9.9869355E+06